# COMUNICACIONES DIGITALES
> Máster en Ingeniería de Telecomunicación<br>
> Curso 2021/22

## Práctica 2d - Sistema completo de comunicaciones

En esta práctica veremos un ejemplo de un sistema completo de comunicaciones, con todos los bloques vistos en clase: codificador de fuente, codificador de canal y modulador.
Se pueden configurar distintos aspectos de su funcionamiento:
 - Ficheros de entrada: Pueden ser texto, imágenes (.bmp) o audio (.wav).
 - Codificador fuente: Admite codificación Huffman, LZW o ninguna.
 - Codificador de canal: Admite códigos de repetición, de paridad, Hamming o niguno.
 - Modulación: Admite modulaciones 2-ASK, 4-ASK y QPSK.
 - Canal: Se puede fijar la densidad espectral de potencia del ruido y el ancho de banda del canal
 - Se pueden especificar las formas de los filtros de transmisión y recepción.
 

### 0 - PARÁMETROS DE CONFIGURACIÓN

In [11]:
import os                           # Para manejar nombres de archivos
import numpy as np                  # Librería matemática
import matplotlib.pyplot as plt     # Para procesar el mapa de colores de la imagen
import matplotlib.image as mpimg    # Para leer y grabar imágenes
import scipy.io.wavfile as wav      # Para leer y escribir archivos .wav

fichero_entrada = "entrada.txt"
fichero_salida = "salida"
codigo_fuente = "huffman"       # "none, "huffman" o "lzw" (lzw sólo si es texto)
codigo_canal = "repeticion"     # "none", "repeticion", "paridad" o "hamming"
modulacion = "2-ASK"            # "2-ASK", "4-ASK", "QPSK"

n = 4       # Tamaño total de la palabra código para la codificación de paridad
k = 3       # Número de repeticiones para el código de repetición
q = 3       # Número de bits de control para Hamming
N0 = 0.3    # Densidad espectral de potencia de ruido

BW = 1e3   # Ancho de banda del canal en Hz

ht = [0.5, 0.5, 0.5, 0.5, 0, 0, 0, 0]   #Filtro transmisor 1
gt = [0, 0, 0, 0, 0.5, 0.5, 0.5, 0.5]   #Filtro transmisor 2
hr = [0, 0, 0, 0, 0.5, 0.5, 0.5, 0.5]   #Filtro receptor 1
gr = [0.5, 0.5, 0.5, 0.5, 0, 0, 0, 0]   #Filtro receptor 2

### 1 - GENERACIÓN DEL MENSAJE
Generamos un mensaje que será la entrada a nuestro sistema

In [12]:
#Leo el mensaje a transmitir
nombre, extension = os.path.splitext(fichero_entrada)
if extension == ".txt":
    fichero = open(fichero_entrada, "r")
    mensaje = fichero.read()
    bitsporpalabra = 8
elif extension == ".wav":
    fs, mensaje = wav.read(fichero_entrada)
    bitsporpalabra = 16
elif extension == ".bmp":
    mensaje = mpimg.imread(fichero_entrada).astype(int)-128
    tamanoX, tamanoY = mensaje.shape
    mensaje = np.reshape(mensaje,-1)
    bitsporpalabra = 8
else:
    print("No conozco este tipo de archivo")
    quit()

print("IMPRIMIR EL TAMAÑO DEL ARCHIVO DE ENTRADA EN BYTES")


IMPRIMIR EL TAMAÑO DEL ARCHIVO DE ENTRADA EN BYTES


### 2 - CODIFICACIÓN FUENTE


In [13]:
print(f"Codificación fuente: {codigo_fuente}")
if codigo_fuente == "huffman":
    from CodFuente import gen_huffman_dic, huffman_cod
    probabilidades, codigos = gen_huffman_dic(mensaje)
    bits_fuente = huffman_cod(mensaje, codigos)
elif codigo_fuente == "lzw":
    from CodFuente import gen_lzw_dic, lzw_cod
    diccionario_lzw = gen_lzw_dic(mensaje)
    bits_fuente, bitspormuestralzw, codigo_lzw = lzw_cod(mensaje, diccionario_lzw)
else:   #En cualquier otro caso no utilizo codificación fuente. 
    if extension == ".txt":
        bits_fuente = ''.join(format(ord(i), '08b') for i in mensaje)
    elif extension == ".wav":
        bits_fuente = ''.join([bin(i)[2:].zfill(bitsporpalabra) if i >=0 else bin(pow(2,bitsporpalabra)+i)[2:].zfill(bitsporpalabra) for i in mensaje]) 

print("IMPRIMIR EL TAMAÑO DEL ARCHIVO TRAS LA CODIFICACIÓN FUENTE EN BYTES")


Codificación fuente: huffman
IMPRIMIR EL TAMAÑO DEL ARCHIVO TRAS LA CODIFICACIÓN FUENTE EN BYTES


### 3 - CODIFICACIÓN DE CANAL
 

In [14]:
print(f"Código de canal: {codigo_canal}")
if codigo_canal == "repeticion":
    from CodCanal import repeticion_cod
    bits_canal = repeticion_cod(bits_fuente,k)
elif codigo_canal == "paridad":
    from CodCanal import paridad_cod
    bits_canal = paridad_cod(bits_fuente,n)
elif codigo_canal == "hamming":
    from CodCanal import hamming_cod
    bits_canal = hamming_cod(bits_fuente, q)
else:   #En cualquier otro caso no utilizo codificación de canal
    bits_canal = bits_fuente

print("IMPRIMIR EL TAMAÑO DEL ARCHIVO TRAS LA CODIFICACIÓN DE CANAL EN BYTES")


Código de canal: repeticion
¡OJO!  HAY QUE IMPLEMENTAR LA FUNCION repeticion_cod
IMPRIMIR EL TAMAÑO DEL ARCHIVO TRAS LA CODIFICACIÓN DE CANAL EN BYTES


### 4 - MODULACIÓN

In [15]:
print(f"Modulación: {modulacion}")
if modulacion == "4-ASK":
    from Moduladores import mod_4ASK
    senal_tx = mod_4ASK(bits_canal,ht)
    Rb = BW
elif modulacion == "QPSK":
    from Moduladores import mod_QPSK
    senal_tx = mod_QPSK(bits_canal, ht, gt)
    Rb = BW
else:   #Por defecto utilizo 2-ASK
    from Moduladores import mod_2ASK
    senal_tx = mod_2ASK(bits_canal, ht)
    Rb = BW/2

tiempo = len(bits_canal)/Rb
print(f"Tiempo necesario para transmitir el mensaje: {tiempo}s")

Modulación: 2-ASK
Tiempo necesario para transmitir el mensaje: 90.022s


### 5 - CANAL DE TRANSMISIÓN

In [16]:
from Canal import canalAWGN
from CodCanal import numErrores
sigma = pow(N0/2, 0.5)
senal_rx = canalAWGN(senal_tx, sigma)

### 6 - DEMODULACIÓN

In [17]:
if modulacion == "4-ASK":
    from Moduladores import dem_4ASK
    bits_rx = dem_4ASK(senal_rx,hr,len(ht))
elif modulacion == "QPSK":
    from Moduladores import dem_QPSK
    bits_rx = dem_QPSK(senal_rx, hr, gr, len(ht))
else:   #Por defecto considero una 2-ASK
    from Moduladores import dem_2ASK
    bits_rx = dem_2ASK(senal_rx, hr, len(ht))

errores = numErrores(bits_canal, bits_rx)
print(f"Número de errores durante la transmisión: {errores} (pe = {errores/len(bits_rx):.4f})")

¡OJO!  HAY QUE IMPLEMENTAR LA FUNCION numErrores
Número de errores durante la transmisión: 0 (pe = 0.0000)


### 7 - DECODIFICACIÓN DE CANAL

In [18]:
if codigo_canal == "repeticion":
    from CodCanal import repeticion_dec
    bits_rx_canal = repeticion_dec(bits_rx,k)
elif codigo_canal == "paridad":
    from CodCanal import paridad_dec
    errores,bits_rx_canal = paridad_dec(bits_rx,n)
elif codigo_canal == "hamming":
    from CodCanal import hamming_dec
    bits_rx_canal = hamming_dec(bits_rx, q)
else:
    bits_rx_canal = bits_rx

errores = numErrores(bits_fuente, bits_rx_canal)
print(f"Número de errores tras la corrección de errores: {errores} (pe = {errores/len(bits_rx_canal):.4f})")

¡OJO!  HAY QUE IMPLEMENTAR LA FUNCION repeticion_dec
¡OJO!  HAY QUE IMPLEMENTAR LA FUNCION numErrores
Número de errores tras la corrección de errores: 0 (pe = 0.0000)


### 8 - DECODIFICACIÓN FUENTE

In [19]:
if codigo_fuente == "huffman":
    from CodFuente import huffman_dec
    salida = huffman_dec(bits_rx_canal, codigos)
    if fichero_entrada.endswith(".txt"):
        salida = "".join(salida)
elif codigo_fuente == "lzw":
    from CodFuente import lzw_dec
    salida = lzw_dec(bits_rx_canal, bitspormuestralzw, diccionario_lzw)
else:
    if extension == ".txt":
        salida = ""
        for i in range(0,len(bits_rx_canal),8):
            salida += chr(int(bits_rx_canal[i:i+8],2))
    elif extension == ".wav":
        salida = []
        for i in range(0,len(bits_rx_canal),16):
            palabra = bits_rx_canal[i:i+16]
            if palabra[0] == '0':
                salida.append(int(palabra,2))
            else:
                aux = ''.join('0' if i=='1' else '1' for i in palabra)
                salida.append(-(int(aux,2)+1))

### 9 - SALIDA A FICHERO

In [20]:
if extension == ".txt":
    f_salida = open(fichero_salida+extension, "w")
    f_salida.write(salida)
elif extension == ".wav":
    salida = np.array(salida, dtype=np.int16)
    wav.write(fichero_salida+extension, fs, salida)
elif extension == ".bmp":
    salida = np.reshape(salida[0:tamanoX*tamanoY],(tamanoX,tamanoY))
    salida += 128
    mpimg.imsave(fichero_salida+extension,salida, cmap=plt.get_cmap('gray'))